# Read and Write Mesh Data

In [9]:
from sigmaepsilon.solid import Structure
from sigmaepsilon.solid.fem import BernoulliFrame
import numpy as np
from numpy import pi as PI


## Import Mesh from JSON-like Data

In [10]:
from sigmaepsilon.solid import BeamSection, get_section

# all units in kN and cm

L = 150.  # length of the console [cm]
F = 1.  # value of the vertical load at the free end [kN]
E = 21000.0  # Young's modulus [kN/cm3]
nu = 0.3  # Poisson's ratio [-]
fy = 23.5  # yield strength [kN/cm2]
density = 7750.0 * 1e-6  # mass density [g/cm3]
weight = density * 1e-3 * 9.81  # [kN/cm3]
alpha = 1.2e-05
nElem = 20  # number of subdivisons to use

d_chs = 10.0  # diameter of the tube
t_chs = 1.0  # thickness of the tube
n_chs = 32  # number of subdivisions along the perimeter when discretizing the section
section = get_section('CHS', d=d_chs, t=t_chs, n=n_chs)
beam = BeamSection(wrap=section)
beam.calculate_geometric_properties()
props = beam.geometric_properties
A = props['A']  # area
Iy = props['Iy']  # second moment of inertia around the y axis
Iz = props['Iz']  # second moment of inertia around the z axis
Ix = props['Ix']  # torsional inertia


In [11]:
from sigmaepsilon.io import json2dict, dict2json

jsonpath = 'console_bernoulli.json'


def solve_dict(d_in=None, jsonpath=None):
    if jsonpath is not None:
        if d_in is not None:
            dict2json(jsonpath, d_in)
        else:
            d_in = json2dict(jsonpath)
    d_out, mesh = BernoulliFrame.from_dict(d_in)
    res = Structure(mesh=mesh).linsolve().nodal_dof_solution()[-1, 2, 2]
    return d_out, res


In [12]:
d_in = {
    "_version_": "0.0.1",

    "materials": {
        "Steel": {
            "E": E,
            "nu": nu,
            "fy": fy,
            "type": "steel",
            "density": density,
            "thermal-coefficient": alpha
        },
        "CustomMaterial": {
            "A": A,
            "Ix": Ix,
            "Iy": Iy,
            "Iz": Iz,
            "fy": fy,
            "type": "steel",
            "density": density,
            "thermal-coefficient": alpha
        }
    },

    "sections": {
        "SteelSection": {
            "type": "CHS",
            "d": d_chs,
            "t": t_chs,
            "material": "Steel"
        },
        "CustomSection": {
            "type": "CHS",
            "d": d_chs,
            "t": t_chs,
        }
    },

    "frames": {
        "0": [
            [1.0, 0.0, 0.0],
            [0.0, 1.0, 0.0],
            [0.0, 0.0, 1.0]
        ]
    },

    "points": {
        "0": {
            "x": [0.0, 0.0, 0.0],
            "fixity": [True, ] * 6
        },
        "1": [L/3, 0.0, 0.0],
        "2": {
            "x": [2*L/3, 0.0, 0.0],
        },
        "3": {
            "x": [L, 0.0, 0.0],
            "mass": 1.0,
            "load": {
                "Fx": [F, 0.0, 0.0, 0.0, 0.0, 0.0],
                "Fy": [0.0, F, 0.0, 0.0, 0.0, 0.0],
                "Fz": [0.0, 0.0, F, 0.0, 0.0, 0.0]
            },
            "fixity": {
                'UX' : False
                }
        }
    },

    "cells": {
        "0": {
            "nodes": [0, 1],
            "section": "SteelSection",
            "density": density,
            "frame": [
                [1.0, 0.0, 0.0],
                [0.0, 1.0, 0.0],
                [0.0, 0.0, 1.0]
            ]
        },
        "1": {
            "nodes": [1, 2],
            "section": "CustomSection",
            "material": "Steel",
            "density": density,
            "frame": "auto"
        },
        "2": {
            "nodes": [2, 3],
            "section": "CustomSection",
            "material": "CustomMaterial",
            "density": density,
            "frame": "auto",
            "load": {
                "Fx": [F, 0.0, 0.0, 0.0, 0.0, 0.0],
                "Fy": [0.0, F, 0.0, 0.0, 0.0, 0.0],
                "Fz": [0.0, 0.0, F, 0.0, 0.0, 0.0]
            },
        }
    },
}


In [14]:
d_out, uz = solve_dict(d_in)
print("Tip displacement : {}".format(uz))


Tip displacement : 0.18544389555161656


In [5]:
# Bernoulli solution
EI = E * Iy
sol_exact = F * L**3 / (3 * EI)
print("Tip displacement : {}".format(sol_exact))


Tip displacement : 0.1872413253058413


In [15]:
jsonpath = 'console_bernoulli.json'
d_out, uz = solve_dict(d_in, jsonpath)
print("Tip displacement : {}".format(uz))

d_out, uz = solve_dict(None, jsonpath)
print("Tip displacement : {}".format(uz))


Tip displacement : 0.18544389555161656
Tip displacement : 0.18544389555161656


In [16]:
from sigmaepsilon.examples import download_bernoulli_console_json_20B2
jsonpath = download_bernoulli_console_json_20B2()
d_out, uz = solve_dict(None, jsonpath)
print("Tip displacement : {}".format(uz))


Tip displacement : 0.18544389555161656


## Write Finite Element Mesh to Parquet files

## Read Finite Element Mesh from Parquet files

In [1]:
from sigmaepsilon.solid.fem.cells import cells_dict
cells = list(cells_dict.values(deep=True, return_address=True))
cells

[(['bernoulli', 'B2'], sigmaepsilon.solid.fem.cells.bernoulli2.Bernoulli2),
 (['bernoulli', 'B3'], sigmaepsilon.solid.fem.cells.bernoulli3.Bernoulli3),
 (['membrane', 'T3'], sigmaepsilon.solid.fem.cells.cst.CSTM),
 (['membrane', 'T6'], sigmaepsilon.solid.fem.cells.lst.LSTM),
 (['membrane', 'Q4'], sigmaepsilon.solid.fem.cells.q4.Q4M),
 (['membrane', 'Q9'], sigmaepsilon.solid.fem.cells.q9.Q9M),
 (['mindlin', 'T3'], sigmaepsilon.solid.fem.cells.cst.CSTP),
 (['mindlin', 'T6'], sigmaepsilon.solid.fem.cells.lst.LSTP),
 (['mindlin', 'Q4'], sigmaepsilon.solid.fem.cells.q4.Q4P),
 (['mindlin', 'Q9'], sigmaepsilon.solid.fem.cells.q9.Q9P),
 (['3d', 'H8'], sigmaepsilon.solid.fem.cells.h8.H8),
 (['3d', 'H27'], sigmaepsilon.solid.fem.cells.h27.H27),
 (['3d', 'TET4'], sigmaepsilon.solid.fem.cells.tet4.TET4),
 (['3d', 'TET10'], sigmaepsilon.solid.fem.cells.tet10.TET10)]

In [2]:
list(cells_dict.keys(deep=True, return_address=True))

[['bernoulli', 'B2'],
 ['bernoulli', 'B3'],
 ['membrane', 'T3'],
 ['membrane', 'T6'],
 ['membrane', 'Q4'],
 ['membrane', 'Q9'],
 ['mindlin', 'T3'],
 ['mindlin', 'T6'],
 ['mindlin', 'Q4'],
 ['mindlin', 'Q9'],
 ['3d', 'H8'],
 ['3d', 'H27'],
 ['3d', 'TET4'],
 ['3d', 'TET10']]